# Automating Restaurant Menu to Excel with AI

### This project converts restaurant menus from images or PDFs into structured Excel files, eliminating manual data entry. The free service helps restaurants quickly generate upload-ready spreadsheets, saving time and costs.

**Workflow:**  


` 
PDF → Image → Excel
`

In [43]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import base64
from IPython.display import display, Image, Markdown
import pandas as pd

In [44]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [45]:
# Set the OpenAI connection and model
MODEL = "gpt-4o"
client = OpenAI(api_key=api_key)

## Defining the System Prompt

### We define a detailed system prompt that instructs the GPT model on how to convert the menu images into a structured Excel format.

In [46]:
# Define the system prompt with detailed instructions
system_prompt = """
Convert the menu image to a structured excel sheet format following the provided template and instructions.
This assistant converts restaurant or cafe menu data into a structured Excel sheet that adheres to a specific template.
The template includes categories, subcategories, item names, prices, descriptions, and more, ensuring data consistency.
This assistant helps users fill out each row correctly, following the detailed instructions provided.

Overview:
- Each row in the Excel spreadsheet represents a unique item, categorized under a category or subcategory.
- Category and subcategory names are repeated for items within the same subcategory.
- Certain columns are left blank when not applicable, such as subcategory details for items directly under a category.
- Item details, including names, prices, and descriptions, must be unique for each entry.
- Uploaded menu content will be appended to the existing menu without deleting any current entries.

Columns Guide:

Column Name                    | Description                               | Accepted Values           | Example
-------------------------------|-------------------------------------------|---------------------------|-----------------------
CategoryTitlePt (Column A)      | Category names in Portuguese              | Text, 256 characters max  | Bebidas
CategoryTitleEn (Column B) (Optional) | English translations of category titles | Text, 256 characters max  | Beverages
SubcategoryTitlePt (Column C) (Optional) | Subcategory titles in Portuguese | Text, 256 characters max or blank | Sucos
SubcategoryTitleEn (Column D) (Optional) | English translations of subcategory titles | Text, 256 characters max or blank | Juices
ItemNamePt (Column E)           | Item names in Portuguese                  | Text, 256 characters max  | Água Mineral
ItemNameEn (Column F) (Optional) | English translations of item names | Text, 256 characters max or blank | Mineral Water
ItemPrice (Column G)          | Price of each item without currency symbol  | Text                      | 2.50 or 2,50
Calories (Column H) (Optional) | Caloric content of each item              | Numeric                   | 150
PortionSize (Column I)        | Portion size for each item in units        | Text                      | 500ml, 1, 2-3
Availability (Column J) (Optional) | Current availability of the item     | Numeric: 1 for Yes, 0 for No | 1
ItemDescriptionPt (Column K) (Optional) | Detailed description in Portuguese | Text, 500 characters max  | Contains essential minerals
ItemDescriptionEn (Column L) (Optional) | Detailed description in English | Text, 500 characters max  | Contains essential minerals

Notes:
- Ensure all data entered follows the specified formats to maintain database integrity.
- Review the data for accuracy and consistency before submitting the Excel sheet.
"""


In [57]:
directory =  f"{os.getcwd()}"
IMAGE_DIR =directory

The next cell defines a function to encode images in base64 format and lists all image files (PNG, JPG, JPEG) in the specified directory.


In [59]:
def encode_image(image_path):
    """
    Encodes an image to base64 format.
    """
    with open(image_path, "rb") as image_file:
        import base64
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

# Process images in the directory
try:
    image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
except FileNotFoundError:
    image_files = []
    print(f"Image directory not found: {IMAGE_DIR}")

image_files

['DimSum Amoreiras 1.PNG',
 'DimSum Amoreiras 2.PNG',
 'DimSum Amoreiras 3.PNG',
 'DimSum Amoreiras 4.PNG',
 'DimSum Amoreiras 5.PNG']

In [61]:
test = image_files[0]
test

'DimSum Amoreiras 1.PNG'

In [62]:
# Adding a flag for the headers
headers_added = False

In [63]:
# Create the PANDAS dataframe
df = pd.DataFrame(columns=['CategoryTitlePt', 'CategoryTitleEn', 'SubcategoryTitlePt', 'SubcategoryTitleEn',
                           'ItemNamePt', 'ItemNameEn', 'ItemPrice', 'Calories', 'PortionSize', 'Availability',
                           'ItemDescriptionPt', 'ItemDescriptionEn'])